In [1]:
import open3d as o3d
import numpy as np
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## define func

## loading bottle model (target)

In [2]:
mesh_target = o3d.io.read_triangle_mesh('../dataset/model/art_melon.STL')
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.4, origin=[0.0,0.0,0.0])
o3d.visualization.draw_geometries([mesh_target, mesh_frame])

## converting from triangle mesh to pcd

In [3]:
pcd_target_bottle = mesh_target.sample_points_uniformly(number_of_points=10000)
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.4, origin=[0.0,0.0,0.0])
o3d.visualization.draw_geometries([pcd_target_bottle, mesh_frame])

## (debug) loading another point cloud (target)

In [4]:
pcd_target = o3d.io.read_point_cloud('../dataset/d405/d405-3.ply')
pcd_target = pcd_target.translate([0.1,0,-0.1]) # This make a cause
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0,0,0])
o3d.visualization.draw_geometries([pcd_target, mesh_frame])

## loading point cloud (source)

In [5]:
pcd_source = o3d.io.read_point_cloud('../dataset/d405/d405-1.ply')
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0.0,0.0,0.0])
o3d.visualization.draw_geometries([pcd_source, mesh_frame])

## cleaning source point cloud

In [6]:
# crop pcd with aabb (Note. z-axis looks backward of camera)
min_bound = np.asarray([-1.0, -1.0, -1.0]) # [x_min, y_min, z_min]
max_bound = np.asarray([1.0, 1.0, 1.0]) # [x_max, y_max, z_max]
aabb = o3d.geometry.AxisAlignedBoundingBox(min_bound, max_bound)
pcd_source_cropped = pcd_source.crop(aabb)
o3d.visualization.draw_geometries([pcd_source_cropped, mesh_frame])
pcd_target_cropped = pcd_target.crop(aabb)
o3d.visualization.draw_geometries([pcd_target_cropped, mesh_frame])

In [7]:
# remove table 
plane_model, plane_inlier_indeces = pcd_source_cropped.segment_plane(distance_threshold=0.005, ransac_n=3, num_iterations=500)
pcd_plane = pcd_source_cropped.select_by_index(plane_inlier_indeces)
pcd_plane.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([pcd_source_cropped, pcd_plane])
pcd_source_removed_plane = pcd_source_cropped.select_by_index(plane_inlier_indeces, invert=True)
o3d.visualization.draw_geometries([pcd_source_removed_plane])

In [8]:
# remove outlier
cl, inlier_indeces = pcd_source_removed_plane.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
pcd_source_removed_outlier = pcd_source_removed_plane.select_by_index(inlier_indeces)
o3d.visualization.draw_geometries([pcd_source_removed_outlier])

In [9]:
pcd_source = pcd_source_removed_outlier
pcd_target = pcd_target_cropped

## keypoint creation

In [10]:
# extract features from point clouds
voxel_size = 0.005
def keypoint_and_feature_extraction(pcd: o3d.geometry.PointCloud, voxel_size):
    keypoints = pcd.voxel_down_sample(voxel_size)
    viewpoint = np.array([0, 0, -1.0], dtype='float64')
    radius_normal = 2.0 * voxel_size
    
    keypoints.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    #keypoints.orient_normals_towards_camera_location(viewpoint)

    radius_feature = 5.0 * voxel_size
    feature = o3d.pipelines.registration.compute_fpfh_feature(
        keypoints,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))

    return keypoints, feature

In [11]:
source_kp, source_feature = keypoint_and_feature_extraction(pcd_source, voxel_size)
target_kp, target_feature = keypoint_and_feature_extraction(pcd_target, voxel_size)

In [12]:
def draw_registration_result(source, target, transformation):
    pcds = list()
    for s in source:
        temp = copy.deepcopy(s)
        pcds.append(temp.transform(transformation))
    pcds += target
    o3d.visualization.draw_geometries(pcds)

In [13]:
source_kp.paint_uniform_color([1, 0, 0])
target_kp.paint_uniform_color([0, 0, 1])
initial_trans = np.identity(4)
initial_trans[0,3] = -3.0
draw_registration_result([source_kp], [target_kp], initial_trans)

In [14]:
# ratio test to remove bad keypoints
np_source_feature = source_feature.data.T
np_target_feature = target_feature.data.T

corrs = o3d.utility.Vector2iVector()
threshold = 0.2
for i, feat in enumerate(np_source_feature):
    distance = np.linalg.norm(np_target_feature - feat, axis=1)
    nearest_idx = np.argmin(distance)
    dist_order = np.argsort(distance)
    ratio = distance[dist_order[0]] - distance[dist_order[1]]
    if ratio < threshold:
        corr = np.array([[i], [nearest_idx]], np.int32)
        corrs.append(corr)

print(f'number of correration set: {len(corrs)}')

number of correration set: 9166


In [15]:
# visualize correspondences
def create_lineset_from_correspondenes(corrs_set, pcd1, pcd2, transformation=np.identity(4)):
    #print(f'corrs_set: {len(corrs_set)}, pcd1: {len(pcd1.points)}, pcd2: {pcd2.points}')
    pcd1_tmp = copy.deepcopy(pcd1)
    pcd1_tmp.transform(transformation)
    corrs = np.asarray(corrs_set)
    np_points1 = np.array(pcd1_tmp.points)
    np_points2 = np.array(pcd2.points)
    points = list()
    lines = list()

    #print(f'np_point1: {len(np_points1)}, np_point2: {len(np_points2)}, corrs: {corrs.shape}')

    for i in range(corrs.shape[0]):
        #print(f'corrs[i, 1]: {corrs[i, 1]}')
        points.append(np_points1[corrs[i, 0]])
        points.append(np_points2[corrs[i, 1]])
        lines.append([2*i, (2*i)+1])

    colors = [np.random.rand(3) for i in range(len(lines))]
    line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(points),
    lines =o3d.utility.Vector2iVector(lines)
    )

    line_set.colors = o3d.utility.Vector3dVector(colors)
    return line_set

In [16]:
line_set = create_lineset_from_correspondenes(corrs, source_kp, target_kp, initial_trans)
draw_registration_result([pcd_source, source_kp], [pcd_target, target_kp, line_set], initial_trans)

## RANSAC pipeline including feature extraction

In [17]:
# estimate transform with RANSAC
distance_threshold = voxel_size*1.5
result = o3d.pipelines.registration.registration_ransac_based_on_correspondence(
    source=source_kp, target=target_kp, corres=corrs, max_correspondence_distance=distance_threshold,
    estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
    ransac_n = 5,
    checkers = [o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.4), 
               o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)],
    criteria = o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.09)
)

In [18]:
result.transformation

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [19]:
result.correspondence_set

std::vector<Eigen::Vector2i> with 0 elements.
Use numpy.asarray() to access data.

In [20]:
line_set = create_lineset_from_correspondenes(result.correspondence_set, source_kp, target_kp, initial_trans)
draw_registration_result([pcd_source, source_kp], [pcd_target, target_kp, line_set], initial_trans)

pcd_source.paint_uniform_color([1,0,0])
pcd_target.paint_uniform_color([1,0,1])
draw_registration_result([source_kp], [target_kp], result.transformation)

[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.


## ICP pipeline

In [21]:
distance_threshold = voxel_size * 1.5

pcd_source = pcd_source.voxel_down_sample(voxel_size)
pcd_target = pcd_target.voxel_down_sample(voxel_size)

pcd_source.estimate_normals()
pcd_target.estimate_normals()

source_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
    pcd_source, o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=100))
target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
    pcd_target, o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=100))

result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    pcd_source, pcd_target, source_fpfh, target_fpfh, True,
    distance_threshold,
    o3d.pipelines.registration.TransformationEstimationPointToPlane(),
    ransac_n=3,
    checkers = [o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
                o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold),
               ],
    criteria = o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999)
)
draw_registration_result([pcd_source], [pcd_target], result.transformation)

## show result

In [22]:
distance_threshold = voxel_size * 1.5

pcd_source = pcd_source.voxel_down_sample(voxel_size)
pcd_target = pcd_target_bottle.voxel_down_sample(voxel_size)

pcd_source.estimate_normals()
pcd_target.estimate_normals()

source_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
    pcd_source, o3d.geometry.KDTreeSearchParamHybrid(radius=0.01, max_nn=30))
target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
    pcd_target, o3d.geometry.KDTreeSearchParamHybrid(radius=0.01, max_nn=30))

result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    pcd_source, pcd_target, source_fpfh, target_fpfh, True,
    distance_threshold,
    o3d.pipelines.registration.TransformationEstimationPointToPlane(),
    ransac_n=3,
    checkers = [o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
                o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold),
               ],
    criteria = o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999)
)
draw_registration_result([pcd_source], [pcd_target], result.transformation)

In [23]:
line_set = create_lineset_from_correspondenes(result.correspondence_set, pcd_source, pcd_target, initial_trans)
draw_registration_result([pcd_source], [pcd_target, line_set], initial_trans)

[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
